In [1]:
# !conda install geoplot -c conda-forge

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
from shapely.geometry import Point, Polygon
# import geoplot
%matplotlib inline
import requests
from datetime import datetime
import datetime
import time
from pandas import crosstab

In [3]:
copper = pd.read_csv('toxics/Copper Samples.csv')
copper

,EPA Region,Submission Year,Submission Quarter,PWS ID,PWS Name,Primacy Agency Code,Contaminant Name,Sample Measure (mg/L),Sampling Start Date,Sampling End Date,...,Owner Type Description,Primacy Type Description,Population Served Count,Primary Source Code,PWS Deactivation Date,Season Begin Date,Season End Date,Is Wholesaler Ind,Is School Or Daycare Ind,Service Connections Count
0,10,2021,3,AK2110619,HAINES,AK,COPPER SUMMARY,1.40,01/01/2013,12/31/2015,...,Local government,State,1613,SW,04-AUG-2016,01-01,12-31,N,N,605
1,10,2021,3,AK2120012,VALLENAR VIEW MOBILE HOME PARK,AK,COPPER SUMMARY,1.70,01/01/2016,06/30/2016,...,Private,State,225,SW,NaN,NaN,NaN,N,N,77
2,10,2021,3,AK2120012,VALLENAR VIEW MOBILE HOME PARK,AK,COPPER SUMMARY,1.64,01/01/2017,06/30/2017,...,Private,State,225,SW,NaN,NaN,NaN,N,N,77
3,10,2021,3,AK2120127,SAXMAN,AK,COPPER SUMMARY,1.86,07/01/2005,12/31/2005,...,Local government,State,450,SW,NaN,NaN,NaN,N,N,127
4,10,2021,3,AK2120135,BOYER TOWING,AK,COPPER SUMMARY,5.32,01/01/2014,06/30/2014,...,Private,State,7,SW,07-FEB-2020,01-01,12-31,N,N,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14066,8,2021,3,WY5601660,WYOMING MEDIUM CORRECTIONAL INSTITUTION,WY,COPPER SUMMARY,1.69,07/01/2014,12/31/2014,...,State government,State,0,GWP,12-AUG-2016,01-01,12-31,N,N,24
14067,8,2021,3,WY5601660,WYOMING MEDIUM CORRECTIONAL INSTITUTION,WY,COPPER SUMMARY,1.69,07/01/2014,12/31/2014,...,State government,State,0,GWP,12-AUG-2016,01-01,12-31,N,N,24
14068,8,2021,3,WY5601660,WYOMING MEDIUM CORRECTIONAL INSTITUTION,WY,COPPER SUMMARY,1.97,01/01/2015,06/30/2015,...,State government,State,0,GWP,12-AUG-2016,01-01,12-31,N,N,24
14069,8,2021,3,WY5601668,JONAH ENERGY LLC FIELD OFFICE,WY,COPPER SUMMARY,2.73,01/01/2018,12/31/2020,...,Private,State,92,GW,NaN,01-01,12-31,N,N,1


In [4]:
copper.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14071 entries, 0 to 14070
Data columns (total 22 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   EPA Region                 14071 non-null  int64  
 1   Submission Year            14071 non-null  int64  
 2   Submission Quarter         14071 non-null  int64  
 3   PWS ID                     14071 non-null  object 
 4   PWS Name                   14071 non-null  object 
 5   Primacy Agency Code        14071 non-null  object 
 6   Contaminant Name           14071 non-null  object 
 7   Sample Measure (mg/L)      14071 non-null  float64
 8   Sampling Start Date        14071 non-null  object 
 9   Sampling End Date          14071 non-null  object 
 10  PWS Activity Code          14071 non-null  object 
 11  PWS Type Code              14071 non-null  object 
 12  Owner Type Description     14071 non-null  object 
 13  Primacy Type Description   14071 non-null  obj

In [5]:
copper['Sampling Start Date'] = pd.to_datetime(copper['Sampling Start Date'])
copper['Sampling End Date'] = pd.to_datetime(copper['Sampling End Date'])

In [6]:
copper['Sample Duration Days'] = copper['Sampling End Date'] - copper['Sampling Start Date']
copper['Sample Duration Days'].value_counts()

183 days     3578
180 days     3294
1095 days    1778
364 days     1748
121 days     1127
181 days      933
1094 days     790
365 days      430
334 days       97
153 days       88
151 days       69
335 days       39
152 days       25
29 days        22
1034 days      14
3286 days       8
729 days        4
122 days        4
91 days         3
730 days        2
60 days         2
182 days        2
3287 days       2
363 days        2
304 days        2
395 days        1
1065 days       1
149 days        1
910 days        1
2190 days       1
272 days        1
1064 days       1
90 days         1
Name: Sample Duration Days, dtype: int64

In [7]:
copper['Sample Start Year'] = pd.DatetimeIndex(copper['Sampling Start Date']).year
copper['Sample End Year'] = pd.DatetimeIndex(copper['Sampling End Date']).year
copper['Sample End Year']

0        2015
1        2016
2        2017
3        2005
4        2014
         ... 
14066    2014
14067    2014
14068    2015
14069    2020
14070    2010
Name: Sample End Year, Length: 14071, dtype: int64

In [8]:
copper.drop(columns=['Sampling Start Date', 'Sampling End Date'], inplace=True)

In [9]:
copper.describe()

,EPA Region,Submission Year,Submission Quarter,Sample Measure (mg/L),Population Served Count,Service Connections Count,Sample Duration Days,Sample Start Year,Sample End Year
count,14071.000000,14071.0,14071.0,14071.000000,1.407100e+04,14071.000000,14071,14071.000000,14071.000000
mean,5.023452,2021.0,3.0,49.977221,1.780232e+03,619.481558,376 days 01:26:10.122947908,2011.384905,2011.759576
std,2.623526,0.0,0.0,3127.167940,2.029585e+04,8612.754176,358 days 02:23:36.840791968,4.919648,4.989738
min,1.000000,2021.0,3.0,0.215000,0.000000e+00,1.000000,29 days 00:00:00,2005.000000,2005.000000
25%,3.000000,2021.0,3.0,1.580000,7.200000e+01,1.000000,180 days 00:00:00,2007.000000,2007.000000
50%,5.000000,2021.0,3.0,1.940000,2.180000e+02,26.000000,183 days 00:00:00,2011.000000,2011.000000
75%,7.000000,2021.0,3.0,2.900000,7.530000e+02,224.000000,364 days 00:00:00,2016.000000,2016.000000
max,10.000000,2021.0,3.0,370000.000000,1.308955e+06,679489.000000,3287 days 00:00:00,2021.000000,2023.000000


In [10]:
copper.isna().sum()

EPA Region                       0
Submission Year                  0
Submission Quarter               0
PWS ID                           0
PWS Name                         0
Primacy Agency Code              0
Contaminant Name                 0
Sample Measure (mg/L)            0
PWS Activity Code                0
PWS Type Code                    0
Owner Type Description           0
Primacy Type Description         0
Population Served Count          0
Primary Source Code              2
PWS Deactivation Date        12184
Season Begin Date             8045
Season End Date               8045
Is Wholesaler Ind                0
Is School Or Daycare Ind         0
Service Connections Count        0
Sample Duration Days             0
Sample Start Year                0
Sample End Year                  0
dtype: int64

In [11]:
copper.drop(columns=['PWS Deactivation Date', 'Season Begin Date', 'Season End Date',
                     'Is Wholesaler Ind', 'Is School Or Daycare Ind'], inplace=True)
copper[copper['Primary Source Code'].isna()]

,EPA Region,Submission Year,Submission Quarter,PWS ID,PWS Name,Primacy Agency Code,Contaminant Name,Sample Measure (mg/L),PWS Activity Code,PWS Type Code,Owner Type Description,Primacy Type Description,Population Served Count,Primary Source Code,Service Connections Count,Sample Duration Days,Sample Start Year,Sample End Year
4181,6,2021,3,LA0000000,LCR DEMO SYSTEM,LA,COPPER SUMMARY,1.50,N,CWS,State government,State,0,NaN,5,180 days,2009,2009
5517,5,2021,3,MI0002843,GREEN LAKE TOWNSHIP,MI,COPPER SUMMARY,1.62,N,CWS,Local government,State,1,NaN,1,180 days,2010,2010


In [12]:
copper['Primary Source Code'].value_counts()

GW     11888
SWP      640
SW       572
GWP      537
GU       402
GUP       30
Name: Primary Source Code, dtype: int64

In [13]:
copper.rename({np.nan: '?'}, inplace=True)

In [14]:
copper['Sample Start Year'].sort_values()

7035     2005
9908     2005
9907     2005
9906     2005
9905     2005
         ... 
3234     2021
10736    2021
549      2021
12623    2021
4432     2021
Name: Sample Start Year, Length: 14071, dtype: int64

In [15]:
copper['Sample End Year'].sort_values()

7035     2005
10395    2005
10394    2005
10393    2005
10392    2005
         ... 
6718     2022
833      2022
10965    2023
6552     2023
1022     2023
Name: Sample End Year, Length: 14071, dtype: int64

In [16]:
# remove = ['AS', 'DC', 'GU', 'PR', 'NN', 'VI', 'MP']
# copper_cln = copper[~copper['Primacy Agency Code'].isin(remove)]
# copper_cln.shape

In [17]:
# https://pandas.pydata.org/docs/user_guide/reshaping.html

copper_by_states_year = copper.groupby(['Primacy Agency Code', 'Sample Start Year']).agg({'Sample Measure (mg/L)': ['mean']})
copper_states_by_year = copper_by_states_year.unstack(0).T
copper_states_by_year

Sample Start Year                                     2005       2006  \
                           Primacy Agency Code                          
Sample Measure (mg/L) mean AK                     2.311458   2.572062   
                           AL                     2.166667        NaN   
                           AR                     1.890000        NaN   
                           AZ                     1.800000   2.098000   
                           CA                     4.757619   2.759518   
                           CO                    10.830057   2.428085   
                           CT                     2.531071   2.203389   
                           DE                     3.111214  25.877000   
                           FL                     1.645000   2.355897   
                           GA                     2.776200   2.520479   
                           HI                     1.450000        NaN   
                           IA                     1.969500   1.930476   
                           ID                     2.126750   1.779667   
                           IL                    13.218767   1.951600   
                           IN                     2.100437   2.442857   
                           KS                     1.795000   1.628571   
                           KY                          NaN        NaN   
                           LA                     2.692000   1.833333   
                           MA                     2.582639   2.662167   
                           MD                     3.102324   3.127576   
                           ME                     2.221154   2.107308   
                           MI                     2.009778   1.714900   
                           MN                     2.164636   2.391938   
                           MO                     3.182200   1.960000   
                           MP                          NaN        NaN   
                           MS                     1.654400   1.853782   
                           MT                     2.237538   2.020882   
                           NC                     7.529233   2.680820   
                           ND                     2.045500   2.298500   
                           NE                     1.805314   1.992125   
                           NH                          NaN   2.847636   
                           NJ                     3.576203   5.429355   
                           NM                     2.575909   2.110000   
                           NN                     2.076667        NaN   
                           NV                     2.042857   1.600000   
                           NY                     2.541855   2.725000   
                           OH                   211.078599   2.386364   
                           OK                     4.571767   2.152636   
                           OR                     1.704235        NaN   
                           PA                     2.383426   2.298294   
                           PR                     1.566667   2.377500   
                           RI                     2.078100   3.724286   
                           SC                     2.622885   2.630209   
                           SD                     1.450000   1.400000   
                           TN                     1.900000        NaN   
                           TX                     2.811000   2.126111   
                           UT                     9.720714   1.560333   
                           VA                     3.457423   1.954261   
                           VI                          NaN        NaN   
                           VT                     2.252000   2.877500   
                           WA                     2.549286   1.972083   
                           WI                     2.085500   1.757353   
                           WV                    15.283386

In [18]:
copper_by_states_year_pop = copper.groupby(['Primacy Agency Code', 'Sample Start Year']).agg({'Population Served Count': ['mean']})
copper_states_by_year_pop = copper_by_states_year.unstack(0).T
copper_states_by_year_pop

Sample Start Year                                     2005       2006  \
                           Primacy Agency Code                          
Sample Measure (mg/L) mean AK                     2.311458   2.572062   
                           AL                     2.166667        NaN   
                           AR                     1.890000        NaN   
                           AZ                     1.800000   2.098000   
                           CA                     4.757619   2.759518   
                           CO                    10.830057   2.428085   
                           CT                     2.531071   2.203389   
                           DE                     3.111214  25.877000   
                           FL                     1.645000   2.355897   
                           GA                     2.776200   2.520479   
                           HI                     1.450000        NaN   
                           IA                     1.969500   1.930476   
                           ID                     2.126750   1.779667   
                           IL                    13.218767   1.951600   
                           IN                     2.100437   2.442857   
                           KS                     1.795000   1.628571   
                           KY                          NaN        NaN   
                           LA                     2.692000   1.833333   
                           MA                     2.582639   2.662167   
                           MD                     3.102324   3.127576   
                           ME                     2.221154   2.107308   
                           MI                     2.009778   1.714900   
                           MN                     2.164636   2.391938   
                           MO                     3.182200   1.960000   
                           MP                          NaN        NaN   
                           MS                     1.654400   1.853782   
                           MT                     2.237538   2.020882   
                           NC                     7.529233   2.680820   
                           ND                     2.045500   2.298500   
                           NE                     1.805314   1.992125   
                           NH                          NaN   2.847636   
                           NJ                     3.576203   5.429355   
                           NM                     2.575909   2.110000   
                           NN                     2.076667        NaN   
                           NV                     2.042857   1.600000   
                           NY                     2.541855   2.725000   
                           OH                   211.078599   2.386364   
                           OK                     4.571767   2.152636   
                           OR                     1.704235        NaN   
                           PA                     2.383426   2.298294   
                           PR                     1.566667   2.377500   
                           RI                     2.078100   3.724286   
                           SC                     2.622885   2.630209   
                           SD                     1.450000   1.400000   
                           TN                     1.900000        NaN   
                           TX                     2.811000   2.126111   
                           UT                     9.720714   1.560333   
                           VA                     3.457423   1.954261   
                           VI                          NaN        NaN   
                           VT                     2.252000   2.877500   
                           WA                     2.549286   1.972083   
                           WI                     2.085500   1.757353   
                           WV                    15.283386

In [19]:
def rename_cols(string, columns):
    cols = []
    for col in columns:
        col = str(col)
        new_col = col+'_'+string
        cols.append(new_col)
    return cols
        
copper_states_by_year.columns = rename_cols('Copper Sample Measure (mg/L)', copper_states_by_year.columns)

In [20]:
copper_states_by_year_pop.columns = rename_cols('Copper Population Served Count', copper_states_by_year_pop.columns)

In [21]:
# https://stackoverflow.com/questions/14507794/pandas-how-to-flatten-a-hierarchical-index-in-columns/55757002#55757002

copper_states_by_year.columns = copper_states_by_year.columns.to_flat_index()
copper_states_by_year_pop.columns = copper_states_by_year_pop.columns.to_flat_index()
copper_states_by_year.index = copper_states_by_year.index.to_flat_index()
copper_states_by_year_pop.index = copper_states_by_year_pop.index.to_flat_index()

In [22]:
copper_states_by_year.index = [list(row)[2] for row in copper_states_by_year.index]

copper_states_by_year_pop.index = [list(row)[2] for row in copper_states_by_year_pop.index]

In [23]:
copper_states_by_year = pd.concat([copper_states_by_year, copper_states_by_year_pop], axis=1)

In [24]:
remove = ['AS', 'DC', 'GU', 'PR', 'NN', 'VI', 'MP']
copper_states_by_year = copper_states_by_year[~copper_states_by_year.index.isin(remove)]

In [25]:
# copper_by_states = copper.groupby(['Primacy Agency Code']).agg({'Sample Measure (mg/L)': ['count', 'mean', 'min', 'max', 'sum'],
#                                            'Population Served Count': ['count', 'mean', 'min', 'max', 'sum'],
#                                            'Sample Start Year': ['median', 'min', 'max'],
#                                            'Sample End Year': ['median', 'min', 'max']})

In [26]:
lead = pd.read_csv('toxics/Lead Completeness System with Samples.csv', encoding="utf8", encoding_errors='ignore')
lead

,EPA Region,Submission Year,Submission Quarter,PWS ID,PWS Name,Primacy Agency Code,Contaminant Name,Sample Measure (mg/L),Sampling Start Date,Sampling End Date,...,Owner Type Description,Primacy Type Description,Population Served Count,Primary Source Code,PWS Deactivation Date,Season Begin Date,Season End Date,Is Wholesaler Ind,Is School Or Daycare Ind,Service Connections Count
0,10,2021,3,AK2110342,CITY AND BOROUGH OF JUNEAU,AK,LEAD SUMMARY,0.001700,01/01/2005,12/31/2007,...,Local government,State,38526,SW,NaN,NaN,NaN,N,N,7657
1,10,2021,3,AK2110342,CITY AND BOROUGH OF JUNEAU,AK,LEAD SUMMARY,0.000000,01/01/2017,12/31/2019,...,Local government,State,38526,SW,NaN,NaN,NaN,N,N,7657
2,10,2021,3,AK2110342,CITY AND BOROUGH OF JUNEAU,AK,LEAD SUMMARY,0.000714,01/01/2008,12/31/2010,...,Local government,State,38526,SW,NaN,NaN,NaN,N,N,7657
3,10,2021,3,AK2110342,CITY AND BOROUGH OF JUNEAU,AK,LEAD SUMMARY,0.001400,01/01/2011,12/31/2013,...,Local government,State,38526,SW,NaN,NaN,NaN,N,N,7657
4,10,2021,3,AK2110342,CITY AND BOROUGH OF JUNEAU,AK,LEAD SUMMARY,0.000000,01/01/2014,12/31/2016,...,Local government,State,38526,SW,NaN,NaN,NaN,N,N,7657
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63825,8,2021,3,WY5680122,USAF F.E. WARREN AFB,WY,LEAD SUMMARY,0.004000,01/01/2005,12/31/2005,...,Federal government,State,5500,SWP,NaN,NaN,NaN,N,N,400
63826,8,2021,3,WY5680122,USAF F.E. WARREN AFB,WY,LEAD SUMMARY,0.000000,01/01/2013,12/31/2015,...,Federal government,State,5500,SWP,NaN,NaN,NaN,N,N,400
63827,8,2021,3,WY5680122,USAF F.E. WARREN AFB,WY,LEAD SUMMARY,0.003000,01/01/2010,12/31/2012,...,Federal government,State,5500,SWP,NaN,NaN,NaN,N,N,400
63828,8,2021,3,WY5680122,USAF F.E. WARREN AFB,WY,LEAD SUMMARY,0.002000,01/01/2007,12/31/2009,...,Federal government,State,5500,SWP,NaN,NaN,NaN,N,N,400


In [27]:
lead['Sampling Start Date'] = pd.to_datetime(lead['Sampling Start Date'])
lead['Sampling End Date'] = pd.to_datetime(lead['Sampling End Date'])

In [28]:
lead.describe()

,EPA Region,Submission Year,Submission Quarter,Sample Measure (mg/L),Population Served Count,Service Connections Count
count,63830.000000,63830.0,63830.0,63830.000000,6.383000e+04,63830.000000
mean,5.154943,2021.0,3.0,0.015131,3.730089e+04,10866.443475
std,2.408720,0.0,0.0,0.246968,1.849273e+05,34636.629158
min,1.000000,2021.0,3.0,-0.005000,3.301000e+03,1.000000
25%,4.000000,2021.0,3.0,0.000100,5.500000e+03,1909.000000
50%,5.000000,2021.0,3.0,0.002000,9.820000e+03,3464.000000
75%,6.000000,2021.0,3.0,0.004500,2.450000e+04,8187.500000
max,10.000000,2021.0,3.0,14.700000,8.271000e+06,936578.000000


In [29]:
lead['Sample Duration Days'] = lead['Sampling End Date'] - lead['Sampling Start Date']
lead['Sample Duration Days'].value_counts()

1095 days    27730
1094 days    11965
364 days      8243
183 days      4318
180 days      3381
365 days      2228
121 days      1877
334 days      1473
181 days      1032
335 days       647
153 days       326
151 days       220
122 days       162
152 days        91
1034 days       41
729 days        25
3286 days       19
730 days        15
182 days         5
1064 days        4
30 days          4
363 days         3
1085 days        2
3287 days        2
548 days         2
1460 days        2
272 days         1
29 days          1
211 days         1
171 days         1
545 days         1
943 days         1
1065 days        1
942 days         1
1279 days        1
184 days         1
1086 days        1
1066 days        1
91 days          1
Name: Sample Duration Days, dtype: int64

In [30]:
lead['Sample Start Year'] = pd.DatetimeIndex(lead['Sampling Start Date']).year
lead['Sample End Year'] = pd.DatetimeIndex(lead['Sampling End Date']).year
lead['Sample End Year']

0        2007
1        2019
2        2010
3        2013
4        2016
         ... 
63825    2005
63826    2015
63827    2012
63828    2009
63829    2006
Name: Sample End Year, Length: 63830, dtype: int64

In [31]:
lead.drop(columns=['Sampling Start Date', 'Sampling End Date'], inplace=True)

In [32]:
lead.isna().sum()

EPA Region                       0
Submission Year                  0
Submission Quarter               0
PWS ID                           0
PWS Name                         0
Primacy Agency Code              0
Contaminant Name                 0
Sample Measure (mg/L)            0
PWS Activity Code                0
PWS Type Code                    0
Owner Type Description           0
Primacy Type Description         0
Population Served Count          0
Primary Source Code              4
PWS Deactivation Date        63099
Season Begin Date            62475
Season End Date              62475
Is Wholesaler Ind                0
Is School Or Daycare Ind         0
Service Connections Count        0
Sample Duration Days             0
Sample Start Year                0
Sample End Year                  0
dtype: int64

In [33]:
lead.drop(columns=['PWS Deactivation Date', 'Season Begin Date', 'Season End Date',
                     'Is Wholesaler Ind', 'Is School Or Daycare Ind'], inplace=True)
lead[lead['Primary Source Code'].isna()]

,EPA Region,Submission Year,Submission Quarter,PWS ID,PWS Name,Primacy Agency Code,Contaminant Name,Sample Measure (mg/L),PWS Activity Code,PWS Type Code,Owner Type Description,Primacy Type Description,Population Served Count,Primary Source Code,Service Connections Count,Sample Duration Days,Sample Start Year,Sample End Year
11348,3,2021,3,DC0000008,CHILDRENS NATIONAL MEDICAL CENTER (CNMC),DC,LEAD SUMMARY,0.0020,N,NTNCWS,Public/Private,State,5000,NaN,2,183 days,2009,2009
11349,3,2021,3,DC0000008,CHILDRENS NATIONAL MEDICAL CENTER (CNMC),DC,LEAD SUMMARY,0.0020,N,NTNCWS,Public/Private,State,5000,NaN,2,180 days,2010,2010
40010,2,2021,3,NY0630144,NORTH CHAUTAUQUA COUNTY WATER DISTRICT,NY,LEAD SUMMARY,0.0035,A,CWS,Local government,State,6200,NaN,3000,183 days,2018,2018
40011,2,2021,3,NY0630144,NORTH CHAUTAUQUA COUNTY WATER DISTRICT,NY,LEAD SUMMARY,0.0016,A,CWS,Local government,State,6200,NaN,3000,180 days,2019,2019


In [34]:
lead.rename({np.nan: '?'}, inplace=True)

In [35]:
lead_by_states_year = lead.groupby(['Primacy Agency Code', 'Sample Start Year']).agg({'Sample Measure (mg/L)': ['mean']})
lead_states_by_year = lead_by_states_year.unstack(0).T
lead_states_by_year

lead_by_states_year_pop = lead.groupby(['Primacy Agency Code', 'Sample Start Year']).agg({'Population Served Count': ['mean']})
lead_states_by_year_pop = lead_by_states_year.unstack(0).T
lead_states_by_year_pop

lead_states_by_year.columns = rename_cols('Lead Sample Measure (mg/L)', lead_states_by_year.columns)
lead_states_by_year_pop.columns = rename_cols('Lead Population Served Count', lead_states_by_year_pop.columns)

lead_states_by_year.columns = lead_states_by_year.columns.to_flat_index()
lead_states_by_year_pop.columns = lead_states_by_year_pop.columns.to_flat_index()
lead_states_by_year.index = lead_states_by_year.index.to_flat_index()
lead_states_by_year_pop.index = lead_states_by_year_pop.index.to_flat_index()

lead_states_by_year.index = [list(row)[2] for row in lead_states_by_year.index]

lead_states_by_year_pop.index = [list(row)[2] for row in lead_states_by_year_pop.index]

lead_states_by_year = pd.concat([lead_states_by_year, lead_states_by_year_pop], axis=1)

remove = ['AS', 'DC', 'GU', 'PR', 'NN', 'VI', 'MP']
lead_states_by_year = lead_states_by_year[~lead_states_by_year.index.isin(remove)]

In [36]:
# lead_by_states = lead.groupby(['Primacy Agency Code']).agg({'Sample Measure (mg/L)': ['count', 'mean', 'min', 'max', 'sum'],
#                                            'Population Served Count': ['count', 'mean', 'min', 'max', 'sum'],
#                                            'Sample Start Year': ['mean', 'min', 'max'],
#                                            'Sample End Year': ['mean', 'min', 'max']})
# lead_by_states.head()

In [37]:
copper_states_by_year.shape

(50, 34)

In [38]:
lead_states_by_year.shape

(50, 34)

In [39]:
# remove = ['AS', 'DC', 'GU', 'PR', 'NN', 'VI', 'MP']
# lead_by_states = lead_by_states[~lead_by_states.index.isin(remove)]

In [40]:
copper_states_by_year.reset_index(inplace=True)
lead_states_by_year.reset_index(inplace=True)

In [41]:
copper_states_by_year.rename(columns={'index': 'state_abrv'}, inplace=True)
lead_states_by_year.rename(columns={'index': 'state_abrv'}, inplace=True)

In [42]:
copper_states_by_year

,state_abrv,2005_Copper Sample Measure (mg/L),2006_Copper Sample Measure (mg/L),2007_Copper Sample Measure (mg/L),2008_Copper Sample Measure (mg/L),2009_Copper Sample Measure (mg/L),2010_Copper Sample Measure (mg/L),2011_Copper Sample Measure (mg/L),2012_Copper Sample Measure (mg/L),2013_Copper Sample Measure (mg/L),...,2012_Copper Population Served Count,2013_Copper Population Served Count,2014_Copper Population Served Count,2015_Copper Population Served Count,2016_Copper Population Served Count,2017_Copper Population Served Count,2018_Copper Population Served Count,2019_Copper Population Served Count,2020_Copper Population Served Count,2021_Copper Population Served Count
0,AK,2.311458,2.572062,1.973000,2.633000,697.876842,2.109029,236.765120,2.448340,2.352111,...,2.448340,2.352111,2.846818,1.816176,2.386591,2.198125,2.242857,2.434063,2.230000,1.853750
1,AL,2.166667,NaN,4.150000,2.350000,2.423333,1.820000,3.460000,3.571000,3.790000,...,3.571000,3.790000,3.165000,765.000000,2.384000,2.152500,3.411667,1.575000,2.860000,1.810000
2,AR,1.890000,NaN,NaN,NaN,41.000000,NaN,NaN,NaN,NaN,...,NaN,NaN,2.450000,NaN,NaN,NaN,NaN,NaN,1.460000,NaN
3,AZ,1.800000,2.098000,1.646667,2.383636,1.922000,2.682000,2.032500,3.296667,23.310000,...,3.296667,23.310000,NaN,1.410000,1.734000,3.150000,3.190000,2.000000,5.860000,1.550000
4,CA,4.757619,2.759518,13.474587,2.094981,2.003000,2.348038,3.661476,6.472511,2.207575,...,6.472511,2.207575,2.088860,2.297957,6.088438,2.027990,2.198656,2.300664,2.144956,2.309687
5,CO,10.830057,2.428085,1.666667,2.915000,2.112500,2.075000,2.151636,1.870000,2.159375,...,1.870000,2.159375,2.124583,1.976923,1.955833,1.769273,2.068750,1.960000,1.872500,2.111250
6,CT,2.531071,2.203389,2.457083,2.038214,2.061053,2.780000,2.411538,2.337273,2.750000,...,2.337273,2.750000,2.925185,2.476250,2.102308,2.708891,2.333637,3.023000,1.886631,2.336692
7,DE,3.111214,25.877000,NaN,1.989000,2.123300,2.846250,3.682000,3.302857,2.592000,...,3.302857,2.592000,2.971429,2.476667,3.229091,3.313333,2.700000,2.250000,3.350000,3.900000
8,FL,1.645000,2.355897,2.152125,2.555231,2.032794,1.870760,2.085455,1.968542,1.974231,...,1.968542,1.974231,2.472895,2.125733,2.709792,1.944092,1.960035,2.293395,1.861278,1.450000
9,GA,2.776200,2.520479,2.446960,2.545769,3.032500,2.066429,2.192000,2.562187,2.332955,...,2.562187,2.332955,2.732237,2.309458,2.626778,2.441471,3.025000,2.292714,1.800667,2.158333


In [43]:
copper_states_by_year.isna().sum()

state_abrv                              0
2005_Copper Sample Measure (mg/L)       2
2006_Copper Sample Measure (mg/L)       6
2007_Copper Sample Measure (mg/L)       2
2008_Copper Sample Measure (mg/L)       3
2009_Copper Sample Measure (mg/L)       1
2010_Copper Sample Measure (mg/L)       3
2011_Copper Sample Measure (mg/L)       3
2012_Copper Sample Measure (mg/L)       3
2013_Copper Sample Measure (mg/L)       4
2014_Copper Sample Measure (mg/L)       3
2015_Copper Sample Measure (mg/L)       3
2016_Copper Sample Measure (mg/L)       3
2017_Copper Sample Measure (mg/L)       3
2018_Copper Sample Measure (mg/L)       3
2019_Copper Sample Measure (mg/L)       3
2020_Copper Sample Measure (mg/L)       2
2021_Copper Sample Measure (mg/L)      11
2005_Copper Population Served Count     2
2006_Copper Population Served Count     6
2007_Copper Population Served Count     2
2008_Copper Population Served Count     3
2009_Copper Population Served Count     1
2010_Copper Population Served Coun

In [44]:
copper_states_by_year = copper_states_by_year.fillna(copper_states_by_year.mean())
lead_states_by_year = lead_states_by_year.fillna(lead_states_by_year.mean())

<ipython-input-44-10604f21ad7b>:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  copper_states_by_year = copper_states_by_year.fillna(copper_states_by_year.mean())
<ipython-input-44-10604f21ad7b>:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  lead_states_by_year = lead_states_by_year.fillna(lead_states_by_year.mean())


In [45]:
def poly_cols(df, new_col_name, col_1, col_2, list_years):
    new_cols = []
    col_names = []
    for year in list_years:
        col_name = f'{year}_'+new_col_name
        col_names.append(col_name)
        col_name = df[f'{year}_{col_1}'] * df[f'{year}_{col_2}']
        new_cols.append(col_name)
    new_cols_df = pd.concat(new_cols, axis=1)
    new_cols_df.columns = col_names
        
    return new_cols_df

list_years = list(range(2005, 2021))
new_cols_df = poly_cols(copper_states_by_year, 'copper_mg/L_x_population_served', 
                        'Copper Sample Measure (mg/L)', 'Copper Population Served Count', list_years)
new_cols_df.index = copper_states_by_year['state_abrv']

In [46]:
copperxpop_states_by_year = new_cols_df

In [47]:
new_cols_df_lead = poly_cols(lead_states_by_year, 'lead_mg/L_x_population_served', 
                        'Lead Sample Measure (mg/L)', 'Lead Population Served Count', list_years)
new_cols_df_lead.index = lead_states_by_year['state_abrv']

In [48]:
leadxpop_states_by_year = new_cols_df_lead

In [49]:
copperxpop_states_by_year.reset_index(inplace=True)
leadxpop_states_by_year.reset_index(inplace=True)
leadxpop_states_by_year

,state_abrv,2005_lead_mg/L_x_population_served,2006_lead_mg/L_x_population_served,2007_lead_mg/L_x_population_served,2008_lead_mg/L_x_population_served,2009_lead_mg/L_x_population_served,2010_lead_mg/L_x_population_served,2011_lead_mg/L_x_population_served,2012_lead_mg/L_x_population_served,2013_lead_mg/L_x_population_served,2014_lead_mg/L_x_population_served,2015_lead_mg/L_x_population_served,2016_lead_mg/L_x_population_served,2017_lead_mg/L_x_population_served,2018_lead_mg/L_x_population_served,2019_lead_mg/L_x_population_served,2020_lead_mg/L_x_population_served
0,AK,3.839868e-05,0.000011,0.000040,4.388179e-05,4.060238e-05,3.154968e-05,3.994240e-05,2.673924e-05,1.940794e-05,1.452318e-05,1.179923e-05,7.810501e-06,4.439066e-06,1.031811e-05,3.887469e-06,5.004808e-06
1,AL,1.669901e-05,0.000021,0.000016,1.936102e-05,2.194802e-05,2.008235e-05,2.028438e-05,1.915712e-05,1.540450e-05,1.622684e-05,1.654256e-05,1.293906e-06,1.656337e-06,1.591977e-06,2.525043e-06,1.874503e-06
2,AR,2.186938e-05,0.000029,0.000023,7.146147e-05,2.079913e-05,4.225000e-05,1.888493e-05,2.844444e-05,2.402500e-04,9.576516e-06,9.506250e-05,5.184000e-05,6.676905e-05,1.506198e-04,4.144141e-05,6.524024e-06
3,AZ,1.403565e-05,0.001774,0.000007,9.431723e-06,1.445596e-05,5.941858e-06,1.358126e-04,1.909822e-05,7.077646e-06,1.760579e-05,1.309965e-05,4.125044e-06,3.585393e-06,6.666724e-06,2.951033e-06,1.707845e-05
4,CA,5.407313e-04,0.000022,0.000052,5.344904e-06,4.744534e-06,4.057132e-06,5.725013e-06,2.375832e-06,2.311946e-06,2.343539e-06,1.191498e-06,1.335598e-06,7.057744e-07,7.185656e-07,1.059184e-06,3.703024e-06
5,CO,2.922517e-05,0.000017,0.000015,1.712603e-05,2.959763e-05,4.038822e-05,2.249748e-05,5.600028e-05,4.153086e-05,1.426989e-05,5.228402e-05,2.575563e-05,1.111530e-05,1.175839e-05,1.108926e-05,9.031774e-06
6,CT,6.539887e-07,0.000008,0.000009,1.446965e-06,2.348556e-06,1.546569e-06,4.642151e-07,1.163005e-06,4.588670e-07,9.850562e-07,1.595279e-06,6.337814e-07,7.180332e-07,8.307866e-07,8.327165e-07,1.449616e-06
7,DE,1.897086e-05,0.000009,0.000061,1.764000e-05,2.828306e-05,3.647876e-05,1.273136e-04,1.865006e-05,1.475840e-05,9.922500e-06,7.910156e-06,1.247242e-05,4.536900e-06,4.231837e-06,6.201563e-07,5.725765e-06
8,FL,4.243600e-06,0.000031,0.000028,1.759514e-05,1.555316e-05,1.459240e-05,8.917878e-06,9.798663e-06,6.276624e-06,5.238705e-06,6.093845e-06,1.026784e-05,7.503048e-06,5.604914e-06,6.172920e-06,4.996388e-06
9,GA,1.083507e-05,0.000012,0.000010,1.444000e-05,7.018009e-06,1.740510e-05,1.781104e-05,9.262355e-06,8.692212e-06,4.818334e-06,4.415835e-06,1.491504e-05,6.404567e-06,6.460312e-06,4.658403e-06,3.729727e-06


In [50]:
copperxpop_year_by_state = copperxpop_states_by_year.T
copperxpop_year_by_state.columns = copperxpop_year_by_state.loc['state_abrv']
copperxpop_year_by_state.drop('state_abrv', axis=0, inplace=True)
copperxpop_year_by_state

state_abrv,AK,AL,AR,AZ,CA,CO,CT,DE,FL,GA,...,SD,TN,TX,UT,VA,VT,WA,WI,WV,WY
2005_copper_mg/L_x_population_served,5.34284,4.694444,3.5721,3.24,22.634942,117.290138,6.406323,9.679654,2.706025,7.707286,...,2.1025,3.61,7.901721,94.492286,11.953774,5.071504,6.498858,4.34931,233.581879,2.8561
2006_copper_mg/L_x_population_served,6.615506,7.971941,7.971941,4.401604,7.614941,5.895595,4.854923,669.619129,5.550249,6.352815,...,1.96,7.971941,4.520348,2.43464,3.819136,8.280006,3.889113,3.088289,2.5281,3.61
2007_copper_mg/L_x_population_served,3.892729,17.2225,57.154079,2.711511,181.564494,2.777778,6.037259,57.154079,4.631642,5.987613,...,2.89,1083.594724,3.071256,2.4964,7.540938,5.784025,3.839098,8.614225,2.205225,2.3409
2008_copper_mg/L_x_population_served,6.932689,5.5225,71506.907092,5.681722,4.388944,8.497225,4.154317,3.956121,6.529204,6.480941,...,5.76,71506.907092,3.2761,12.925332,7.197722,3.956483,4.760226,5.153922,244.792101,3.324544
2009_copper_mg/L_x_population_served,487032.086747,5.872544,1681.0,3.694084,4.012009,4.462656,4.247938,4.508403,4.132252,9.196056,...,4.0,2.873025,2.611456,5.9049,26461.135069,3.930306,4.420195,6.306098,4.2849,4.030056
2010_copper_mg/L_x_population_served,4.448005,3.3124,7.125129,7.193124,5.513285,4.305625,7.7284,8.101139,3.499743,4.270127,...,7.125129,123.765625,4.629669,3.093598,6.464912,4.757852,4.597451,6.71633,27.6676,3.074178
2011_copper_mg/L_x_population_served,56057.722049,11.9716,67.197458,4.131056,13.406408,4.629539,5.815518,13.557124,4.349121,4.804864,...,67.197458,2.56,4.393216,303.354519,40.563251,4.314852,2.602486,2.876077,11.441306,7.004844
2012_copper_mg/L_x_population_served,5.994369,12.752041,52.741406,10.868011,41.893394,3.4969,5.462844,10.908865,3.875156,6.564805,...,2.25,52.741406,4.706296,2.373169,1122.327721,7.9524,3.234988,4.3681,33977.5489,4.212756
2013_copper_mg/L_x_population_served,5.532427,14.3641,54.896331,543.3561,4.873387,4.6629,7.5625,6.718464,3.897587,5.442677,...,54.896331,3.24,3.699769,8127.473256,8.679822,3.845521,4.506357,17.597295,973.939264,54.896331
2014_copper_mg/L_x_population_served,8.104374,10.017225,6.0025,5.472059,4.363338,4.513854,8.556708,8.829388,6.115208,7.465118,...,5.472059,3.857296,4.257819,3.084266,4.3264,11.484125,3.050437,4.459488,5.354596,3.378244


In [51]:
copperxpop_pct_change = copperxpop_year_by_state.pct_change().drop(index=['2005_copper_mg/L_x_population_served'])
copperxpop_pct_change

state_abrv,AK,AL,AR,AZ,CA,CO,CT,DE,FL,GA,...,SD,TN,TX,UT,VA,VT,WA,WI,WV,WY
2006_copper_mg/L_x_population_served,0.238200,0.698165,1.231724,0.358520,-0.663576,-0.949735,-0.242167,68.178000,1.051071,-0.175739,...,-0.067776,1.208294,-0.427929,-0.974235,-0.680508,0.632653,-0.401570,-0.289936,-0.989177,0.263961
2007_copper_mg/L_x_population_served,-0.411575,1.160390,6.169406,-0.383972,22.843191,-0.528838,0.243533,-0.914647,-0.165507,-0.057487,...,0.474490,134.926087,-0.320571,0.025367,0.974514,-0.301447,-0.012860,1.789319,-0.127714,-0.351551
2008_copper_mg/L_x_population_served,0.780933,-0.679344,1250.125180,1.095408,-0.975827,2.059001,-0.311887,-0.930781,0.409695,0.082391,...,0.993080,64.990453,0.066697,4.177589,-0.045514,-0.315964,0.239933,-0.401696,110.005499,0.420199
2009_copper_mg/L_x_population_served,70250.541176,0.063385,-0.976492,-0.349830,-0.085883,-0.474810,0.022536,0.139602,-0.367112,0.418938,...,-0.305556,-0.999960,-0.202877,-0.543153,3675.320566,-0.006616,-0.071432,0.223553,-0.982496,0.212213
2010_copper_mg/L_x_population_served,-0.999991,-0.435951,-0.995761,0.947201,0.374195,-0.035188,0.819330,0.796898,-0.153066,-0.535657,...,0.781282,42.078506,0.772831,-0.476096,-0.999756,0.210555,0.040101,0.065053,5.457000,-0.237187
2011_copper_mg/L_x_population_served,12601.890801,2.614177,8.431051,-0.425694,1.431655,0.075230,-0.247513,0.673484,0.242697,0.125227,...,8.431051,-0.979316,-0.051074,97.058807,5.274371,-0.093109,-0.433928,-0.571778,-0.586473,1.278607
2012_copper_mg/L_x_population_served,-0.999893,0.065191,-0.215128,1.630807,2.124878,-0.244655,-0.060644,-0.195341,-0.108979,0.366283,...,-0.966517,19.602112,0.071265,-0.992177,26.668584,0.843030,0.243038,0.518770,2968.726372,-0.398594
2013_copper_mg/L_x_population_served,-0.077063,0.126416,0.040858,48.995910,-0.883672,0.333438,0.384352,-0.384128,0.005788,-0.170931,...,23.398369,-0.938568,-0.213868,3423.734877,-0.992266,-0.516433,0.393006,3.028593,-0.971336,12.030977
2014_copper_mg/L_x_population_served,0.464886,-0.302621,-0.890658,-0.989929,-0.104660,-0.031964,0.131466,0.314197,0.568973,0.371589,...,-0.900320,0.190523,0.150834,-0.999621,-0.501557,1.986364,-0.323081,-0.746581,-0.994502,-0.938461
2015_copper_mg/L_x_population_served,-0.592998,58420.868332,145.848000,-0.636682,0.210222,-0.134171,-0.283391,-0.305288,-0.261065,-0.285531,...,160.082909,-0.250770,0.353830,3.986767,0.227101,-0.579267,0.455941,0.080140,6390.705369,0.284773


In [52]:
copperxpop_pct_change.isna().sum()

state_abrv
AK    0
AL    0
AR    0
AZ    0
CA    0
CO    0
CT    0
DE    0
FL    0
GA    0
HI    0
IA    0
ID    0
IL    0
IN    0
KS    0
KY    0
LA    0
MA    0
MD    0
ME    0
MI    0
MN    0
MO    0
MS    0
MT    0
NC    0
ND    0
NE    0
NH    0
NJ    0
NM    0
NV    0
NY    0
OH    0
OK    0
OR    0
PA    0
RI    0
SC    0
SD    0
TN    0
TX    0
UT    0
VA    0
VT    0
WA    0
WI    0
WV    0
WY    0
dtype: int64

In [53]:
copperxpop_pct_change.fillna(0.0, inplace=True)
copperxpop_pct_change.replace('inf', 0.0, inplace=True)

In [54]:
leadxpop_year_by_state = leadxpop_states_by_year.T
leadxpop_year_by_state.columns = leadxpop_year_by_state.loc['state_abrv']
leadxpop_year_by_state.drop('state_abrv', axis=0, inplace=True)
leadxpop_year_by_state

state_abrv,AK,AL,AR,AZ,CA,CO,CT,DE,FL,GA,...,SD,TN,TX,UT,VA,VT,WA,WI,WV,WY
2005_lead_mg/L_x_population_served,0.000038,0.000017,0.000022,0.000014,0.000541,0.000029,0.000001,0.000019,0.000004,0.000011,...,0.000007,0.000014,0.000014,0.000018,0.000039,0.000036,0.000014,0.000041,0.000004,0.000023
2006_lead_mg/L_x_population_served,0.000011,0.000021,0.000029,0.001774,0.000022,0.000017,0.000008,0.000009,0.000031,0.000012,...,0.00001,0.000012,0.00002,0.000003,0.00002,0.000032,0.000015,0.000134,0.0,0.000022
2007_lead_mg/L_x_population_served,0.00004,0.000016,0.000023,0.000007,0.000052,0.000015,0.000009,0.000061,0.000028,0.00001,...,0.000005,0.000047,0.000007,0.000007,0.000012,0.000225,0.000017,0.00009,0.0,0.000021
2008_lead_mg/L_x_population_served,0.000044,0.000019,0.000071,0.000009,0.000005,0.000017,0.000001,0.000018,0.000018,0.000014,...,0.000012,0.000019,0.00001,0.000014,0.000142,0.000025,0.000019,0.000038,0.000003,0.00008
2009_lead_mg/L_x_population_served,0.000041,0.000022,0.000021,0.000014,0.000005,0.00003,0.000002,0.000028,0.000016,0.000007,...,0.000009,0.000013,0.000011,0.000107,0.00009,0.000036,0.000013,0.000116,0.000003,0.000056
2010_lead_mg/L_x_population_served,0.000032,0.00002,0.000042,0.000006,0.000004,0.00004,0.000002,0.000036,0.000015,0.000017,...,0.000007,0.000005,0.000008,0.000011,0.000011,0.000016,0.000015,0.000102,0.000004,0.000015
2011_lead_mg/L_x_population_served,0.00004,0.00002,0.000019,0.000136,0.000006,0.000022,0.0,0.000127,0.000009,0.000018,...,0.000015,0.000006,0.000007,0.000009,0.000008,0.000013,0.000014,0.000067,0.000055,0.000012
2012_lead_mg/L_x_population_served,0.000027,0.000019,0.000028,0.000019,0.000002,0.000056,0.000001,0.000019,0.00001,0.000009,...,0.000007,0.000006,0.000007,0.000011,0.000009,0.000006,0.000008,0.000074,0.000012,0.000012
2013_lead_mg/L_x_population_served,0.000019,0.000015,0.00024,0.000007,0.000002,0.000042,0.0,0.000015,0.000006,0.000009,...,0.000008,0.000007,0.000012,0.000015,0.000027,0.000009,0.000011,0.000028,0.000216,0.000051
2014_lead_mg/L_x_population_served,0.000015,0.000016,0.00001,0.000018,0.000002,0.000014,0.000001,0.00001,0.000005,0.000005,...,0.000015,0.000006,0.000008,0.000014,0.000007,0.000017,0.000009,0.000033,0.000005,0.00001


In [55]:
leadxpop_pct_change = leadxpop_year_by_state.pct_change().drop(index=['2005_lead_mg/L_x_population_served'])
leadxpop_pct_change

state_abrv,AK,AL,AR,AZ,CA,CO,CT,DE,FL,GA,...,SD,TN,TX,UT,VA,VT,WA,WI,WV,WY
2006_lead_mg/L_x_population_served,-0.706767,0.237309,0.327331,125.427550,-0.959616,-0.407260,11.022438,-0.538155,6.258578,0.143411,...,0.536604,-0.108461,0.478196,-0.854671,-0.498786,-0.122293,0.026642,2.292742,-1.000000,-0.009546
2007_lead_mg/L_x_population_served,2.575480,-0.216858,-0.210998,-0.995944,1.393800,-0.129889,0.182034,5.954415,-0.101912,-0.168513,...,-0.460410,2.876615,-0.645110,1.868789,-0.409401,6.120831,0.155870,-0.330357,NaN,-0.042801
2008_lead_mg/L_x_population_served,0.089986,0.196516,2.120171,0.310524,-0.897751,0.136215,-0.844308,-0.710496,-0.363955,0.401773,...,1.149641,-0.599170,0.386540,0.938881,11.242767,-0.888889,0.142231,-0.570387,inf,2.730814
2009_lead_mg/L_x_population_served,-0.074733,0.133619,-0.708946,0.532695,-0.112326,0.728225,0.623091,0.603348,-0.116054,-0.513988,...,-0.231006,-0.318927,0.153330,6.510911,-0.364193,0.440000,-0.344313,2.015511,-0.113781,-0.297500
2010_lead_mg/L_x_population_served,-0.222960,-0.085004,1.031335,-0.588968,-0.144883,0.364576,-0.341481,0.289774,-0.061773,1.480063,...,-0.194609,-0.589760,-0.304355,-0.899642,-0.879152,-0.555556,0.227085,-0.120308,0.443541,-0.734626
2011_lead_mg/L_x_population_served,0.266016,0.010060,-0.553020,21.856927,0.411099,-0.442969,-0.699842,2.490075,-0.388868,0.023323,...,1.129217,0.183875,-0.160476,-0.151820,-0.230243,-0.186082,-0.125230,-0.343637,14.164088,-0.197860
2012_lead_mg/L_x_population_served,-0.330555,-0.055573,0.506198,-0.859378,-0.585008,1.489180,1.505314,-0.853511,0.098766,-0.479965,...,-0.515921,-0.045022,0.069339,0.216461,0.120020,-0.520068,-0.372776,0.103776,-0.786406,-0.009844
2013_lead_mg/L_x_population_served,-0.274178,-0.195886,7.446289,-0.629408,-0.026890,-0.258381,-0.605447,-0.208667,-0.359441,-0.061555,...,0.081173,0.235198,0.722453,0.343157,1.877454,0.440000,0.245941,-0.618077,17.455545,3.317760
2014_lead_mg/L_x_population_served,-0.251689,0.053383,-0.960139,1.487521,0.013665,-0.656403,1.146714,-0.327671,-0.165363,-0.445672,...,0.850943,-0.148620,-0.339093,-0.062182,-0.748262,0.877915,-0.105121,0.163645,-0.977542,-0.811644
2015_lead_mg/L_x_population_served,-0.187559,0.019456,8.926627,-0.255947,-0.491581,2.663940,0.619480,-0.202806,0.163235,-0.083535,...,-0.531345,-0.370594,0.090221,0.072249,-0.343609,-0.421578,-0.067226,1.670057,-0.487221,-0.046370


In [56]:
leadxpop_pct_change.fillna(0.0, inplace=True)
leadxpop_pct_change.replace(np.inf, 0.0, inplace=True)

In [57]:
copperxpop_pct_change = copperxpop_pct_change.T
copperxpop_pct_change

,2006_copper_mg/L_x_population_served,2007_copper_mg/L_x_population_served,2008_copper_mg/L_x_population_served,2009_copper_mg/L_x_population_served,2010_copper_mg/L_x_population_served,2011_copper_mg/L_x_population_served,2012_copper_mg/L_x_population_served,2013_copper_mg/L_x_population_served,2014_copper_mg/L_x_population_served,2015_copper_mg/L_x_population_served,2016_copper_mg/L_x_population_served,2017_copper_mg/L_x_population_served,2018_copper_mg/L_x_population_served,2019_copper_mg/L_x_population_served,2020_copper_mg/L_x_population_served
state_abrv,,,,,,,,,,,,,,,
AK,0.238200,-0.411575,7.809329e-01,70250.541176,-0.999991,12601.890801,-0.999893,-0.077063,0.464886,-0.592998,0.726791,-0.151701,0.041114,0.177769,-0.160644
AL,0.698165,1.160390,-6.793439e-01,0.063385,-0.435951,2.614177,0.065191,0.126416,-0.302621,58420.868332,-0.999990,-0.184782,1.512158,-0.786878,2.297395
AR,1.231724,6.169406,1.250125e+03,-0.976492,-0.995761,8.431051,-0.215128,0.040858,-0.890658,145.848000,-0.989576,1.608731,-0.673369,-0.099336,-0.697700
AZ,0.358520,-0.383972,1.095408e+00,-0.349830,0.947201,-0.425694,1.630807,48.995910,-0.989929,-0.636682,0.512377,2.300068,0.025558,-0.606922,7.584900
CA,-0.663576,22.843191,-9.758271e-01,-0.085883,0.374195,1.431655,2.124878,-0.883672,-0.104660,0.210222,6.019849,-0.889052,0.175392,0.094944,-0.130779
CO,-0.949735,-0.528838,2.059001e+00,-0.474810,-0.035188,0.075230,-0.244655,0.333438,-0.031964,-0.134171,-0.021222,-0.181675,0.367182,-0.102373,-0.087293
CT,-0.242167,0.243533,-3.118868e-01,0.022536,0.819330,-0.247513,-0.060644,0.384352,0.131466,-0.283391,-0.279219,0.660315,-0.257864,0.678069,-0.610509
DE,68.178000,-0.914647,-9.307815e-01,0.139602,0.796898,0.673484,-0.195341,-0.384128,0.314197,-0.305288,0.699908,0.052858,-0.335955,-0.305556,1.216790
FL,1.051071,-0.165507,4.096954e-01,-0.367112,-0.153066,0.242697,-0.108979,0.005788,0.568973,-0.261065,0.625003,-0.485291,0.016469,0.369084,-0.341335


In [58]:
leadxpop_pct_change = leadxpop_pct_change.T
leadxpop_pct_change

,2006_lead_mg/L_x_population_served,2007_lead_mg/L_x_population_served,2008_lead_mg/L_x_population_served,2009_lead_mg/L_x_population_served,2010_lead_mg/L_x_population_served,2011_lead_mg/L_x_population_served,2012_lead_mg/L_x_population_served,2013_lead_mg/L_x_population_served,2014_lead_mg/L_x_population_served,2015_lead_mg/L_x_population_served,2016_lead_mg/L_x_population_served,2017_lead_mg/L_x_population_served,2018_lead_mg/L_x_population_served,2019_lead_mg/L_x_population_served,2020_lead_mg/L_x_population_served
state_abrv,,,,,,,,,,,,,,,
AK,-0.706767,2.575480,0.089986,-0.074733,-0.222960,0.266016,-0.330555,-0.274178,-0.251689,-0.187559,-0.338050,-0.431654,1.324388,-0.623238,0.287421
AL,0.237309,-0.216858,0.196516,0.133619,-0.085004,0.010060,-0.055573,-0.195886,0.053383,0.019456,-0.921783,0.280106,-0.038857,0.586106,-0.257635
AR,0.327331,-0.210998,2.120171,-0.708946,1.031335,-0.553020,0.506198,7.446289,-0.960139,8.926627,-0.454675,0.287983,1.255833,-0.724861,-0.842572
AZ,125.427550,-0.995944,0.310524,0.532695,-0.588968,21.856927,-0.859378,-0.629408,1.487521,-0.255947,-0.685103,-0.130823,0.859412,-0.557349,4.787280
CA,-0.959616,1.393800,-0.897751,-0.112326,-0.144883,0.411099,-0.585008,-0.026890,0.013665,-0.491581,0.120940,-0.471567,0.018124,0.474026,2.496110
CO,-0.407260,-0.129889,0.136215,0.728225,0.364576,-0.442969,1.489180,-0.258381,-0.656403,2.663940,-0.507390,-0.568432,0.057855,-0.056906,-0.185539
CT,11.022438,0.182034,-0.844308,0.623091,-0.341481,-0.699842,1.505314,-0.605447,1.146714,0.619480,-0.602714,0.132935,0.157031,0.002323,0.740828
DE,-0.538155,5.954415,-0.710496,0.603348,0.289774,2.490075,-0.853511,-0.208667,-0.327671,-0.202806,0.576760,-0.636245,-0.067240,-0.853455,8.232779
FL,6.258578,-0.101912,-0.363955,-0.116054,-0.061773,-0.388868,0.098766,-0.359441,-0.165363,0.163235,0.684953,-0.269268,-0.252982,0.101341,-0.190596


In [59]:
copperxpop_pct_change.reset_index(inplace=True)
copperxpop_pct_change.rename({'index': 'state_abrv'}, axis=1, inplace=True)
copperxpop_pct_change

,state_abrv,2006_copper_mg/L_x_population_served,2007_copper_mg/L_x_population_served,2008_copper_mg/L_x_population_served,2009_copper_mg/L_x_population_served,2010_copper_mg/L_x_population_served,2011_copper_mg/L_x_population_served,2012_copper_mg/L_x_population_served,2013_copper_mg/L_x_population_served,2014_copper_mg/L_x_population_served,2015_copper_mg/L_x_population_served,2016_copper_mg/L_x_population_served,2017_copper_mg/L_x_population_served,2018_copper_mg/L_x_population_served,2019_copper_mg/L_x_population_served,2020_copper_mg/L_x_population_served
0,AK,0.238200,-0.411575,7.809329e-01,70250.541176,-0.999991,12601.890801,-0.999893,-0.077063,0.464886,-0.592998,0.726791,-0.151701,0.041114,0.177769,-0.160644
1,AL,0.698165,1.160390,-6.793439e-01,0.063385,-0.435951,2.614177,0.065191,0.126416,-0.302621,58420.868332,-0.999990,-0.184782,1.512158,-0.786878,2.297395
2,AR,1.231724,6.169406,1.250125e+03,-0.976492,-0.995761,8.431051,-0.215128,0.040858,-0.890658,145.848000,-0.989576,1.608731,-0.673369,-0.099336,-0.697700
3,AZ,0.358520,-0.383972,1.095408e+00,-0.349830,0.947201,-0.425694,1.630807,48.995910,-0.989929,-0.636682,0.512377,2.300068,0.025558,-0.606922,7.584900
4,CA,-0.663576,22.843191,-9.758271e-01,-0.085883,0.374195,1.431655,2.124878,-0.883672,-0.104660,0.210222,6.019849,-0.889052,0.175392,0.094944,-0.130779
5,CO,-0.949735,-0.528838,2.059001e+00,-0.474810,-0.035188,0.075230,-0.244655,0.333438,-0.031964,-0.134171,-0.021222,-0.181675,0.367182,-0.102373,-0.087293
6,CT,-0.242167,0.243533,-3.118868e-01,0.022536,0.819330,-0.247513,-0.060644,0.384352,0.131466,-0.283391,-0.279219,0.660315,-0.257864,0.678069,-0.610509
7,DE,68.178000,-0.914647,-9.307815e-01,0.139602,0.796898,0.673484,-0.195341,-0.384128,0.314197,-0.305288,0.699908,0.052858,-0.335955,-0.305556,1.216790
8,FL,1.051071,-0.165507,4.096954e-01,-0.367112,-0.153066,0.242697,-0.108979,0.005788,0.568973,-0.261065,0.625003,-0.485291,0.016469,0.369084,-0.341335
9,GA,-0.175739,-0.057487,8.239138e-02,0.418938,-0.535657,0.125227,0.366283,-0.170931,0.371589,-0.285531,0.293679,-0.136114,0.535139,-0.425554,-0.383168


In [60]:
leadxpop_pct_change.reset_index(inplace=True)
leadxpop_pct_change.rename({'index': 'state_abrv'}, axis=1, inplace=True)
leadxpop_pct_change

,state_abrv,2006_lead_mg/L_x_population_served,2007_lead_mg/L_x_population_served,2008_lead_mg/L_x_population_served,2009_lead_mg/L_x_population_served,2010_lead_mg/L_x_population_served,2011_lead_mg/L_x_population_served,2012_lead_mg/L_x_population_served,2013_lead_mg/L_x_population_served,2014_lead_mg/L_x_population_served,2015_lead_mg/L_x_population_served,2016_lead_mg/L_x_population_served,2017_lead_mg/L_x_population_served,2018_lead_mg/L_x_population_served,2019_lead_mg/L_x_population_served,2020_lead_mg/L_x_population_served
0,AK,-0.706767,2.575480,0.089986,-0.074733,-0.222960,0.266016,-0.330555,-0.274178,-0.251689,-0.187559,-0.338050,-0.431654,1.324388,-0.623238,0.287421
1,AL,0.237309,-0.216858,0.196516,0.133619,-0.085004,0.010060,-0.055573,-0.195886,0.053383,0.019456,-0.921783,0.280106,-0.038857,0.586106,-0.257635
2,AR,0.327331,-0.210998,2.120171,-0.708946,1.031335,-0.553020,0.506198,7.446289,-0.960139,8.926627,-0.454675,0.287983,1.255833,-0.724861,-0.842572
3,AZ,125.427550,-0.995944,0.310524,0.532695,-0.588968,21.856927,-0.859378,-0.629408,1.487521,-0.255947,-0.685103,-0.130823,0.859412,-0.557349,4.787280
4,CA,-0.959616,1.393800,-0.897751,-0.112326,-0.144883,0.411099,-0.585008,-0.026890,0.013665,-0.491581,0.120940,-0.471567,0.018124,0.474026,2.496110
5,CO,-0.407260,-0.129889,0.136215,0.728225,0.364576,-0.442969,1.489180,-0.258381,-0.656403,2.663940,-0.507390,-0.568432,0.057855,-0.056906,-0.185539
6,CT,11.022438,0.182034,-0.844308,0.623091,-0.341481,-0.699842,1.505314,-0.605447,1.146714,0.619480,-0.602714,0.132935,0.157031,0.002323,0.740828
7,DE,-0.538155,5.954415,-0.710496,0.603348,0.289774,2.490075,-0.853511,-0.208667,-0.327671,-0.202806,0.576760,-0.636245,-0.067240,-0.853455,8.232779
8,FL,6.258578,-0.101912,-0.363955,-0.116054,-0.061773,-0.388868,0.098766,-0.359441,-0.165363,0.163235,0.684953,-0.269268,-0.252982,0.101341,-0.190596
9,GA,0.143411,-0.168513,0.401773,-0.513988,1.480063,0.023323,-0.479965,-0.061555,-0.445672,-0.083535,2.377627,-0.570597,0.008704,-0.278920,-0.199355


In [61]:
states = {"AL":"Alabama", "AK":"Alaska", "AZ":"Arizona", "AR":"Arkansas", "CA":"California", "CO":"Colorado", "CT":"Connecticut", 
          "DE":"Delaware", "FL":"Florida", "GA":"Georgia", "HI":"Hawaii", "ID":"Idaho", "IL":"Illinois", 
          "IN":"Indiana", "IA":"Iowa", "KS":"Kansas", "KY":"Kentucky", "LA":"Louisiana", "ME":"Maine", "MD":"Maryland",
          "MA":"Massachusetts", "MI":"Michigan", "MN":"Minnesota", "MS":"Mississippi", "MO":"Missouri", "MT":"Montana",
          "NE":"Nebraska", "NV":"Nevada", "NH":"New Hampshire", "NJ":"New Jersey", "NM":"New Mexico", "NY":"New York", 
          "NC":"North Carolina", "ND":"North Dakota", "OH":"Ohio", "OK":"Oklahoma", "OR":"Oregon", "PA":"Pennsylvania", 
          "RI":"Rhode Island", "SC":"South Carolina", "SD":"South Dakota", "TN":"Tennessee", "TX":"Texas", "UT":"Utah", "VT":"Vermont",
          "VA":"Virginia", "WA":"Washington", "WV":"West Virginia","WI":"Wisconsin", "WY":"Wyoming"}
 
copperxpop_pct_change['State'] = copperxpop_pct_change['state_abrv'].map(states)
leadxpop_pct_change['State'] = leadxpop_pct_change['state_abrv'].map(states)
copperxpop_pct_change

,state_abrv,2006_copper_mg/L_x_population_served,2007_copper_mg/L_x_population_served,2008_copper_mg/L_x_population_served,2009_copper_mg/L_x_population_served,2010_copper_mg/L_x_population_served,2011_copper_mg/L_x_population_served,2012_copper_mg/L_x_population_served,2013_copper_mg/L_x_population_served,2014_copper_mg/L_x_population_served,2015_copper_mg/L_x_population_served,2016_copper_mg/L_x_population_served,2017_copper_mg/L_x_population_served,2018_copper_mg/L_x_population_served,2019_copper_mg/L_x_population_served,2020_copper_mg/L_x_population_served,State
0,AK,0.238200,-0.411575,7.809329e-01,70250.541176,-0.999991,12601.890801,-0.999893,-0.077063,0.464886,-0.592998,0.726791,-0.151701,0.041114,0.177769,-0.160644,Alaska
1,AL,0.698165,1.160390,-6.793439e-01,0.063385,-0.435951,2.614177,0.065191,0.126416,-0.302621,58420.868332,-0.999990,-0.184782,1.512158,-0.786878,2.297395,Alabama
2,AR,1.231724,6.169406,1.250125e+03,-0.976492,-0.995761,8.431051,-0.215128,0.040858,-0.890658,145.848000,-0.989576,1.608731,-0.673369,-0.099336,-0.697700,Arkansas
3,AZ,0.358520,-0.383972,1.095408e+00,-0.349830,0.947201,-0.425694,1.630807,48.995910,-0.989929,-0.636682,0.512377,2.300068,0.025558,-0.606922,7.584900,Arizona
4,CA,-0.663576,22.843191,-9.758271e-01,-0.085883,0.374195,1.431655,2.124878,-0.883672,-0.104660,0.210222,6.019849,-0.889052,0.175392,0.094944,-0.130779,California
5,CO,-0.949735,-0.528838,2.059001e+00,-0.474810,-0.035188,0.075230,-0.244655,0.333438,-0.031964,-0.134171,-0.021222,-0.181675,0.367182,-0.102373,-0.087293,Colorado
6,CT,-0.242167,0.243533,-3.118868e-01,0.022536,0.819330,-0.247513,-0.060644,0.384352,0.131466,-0.283391,-0.279219,0.660315,-0.257864,0.678069,-0.610509,Connecticut
7,DE,68.178000,-0.914647,-9.307815e-01,0.139602,0.796898,0.673484,-0.195341,-0.384128,0.314197,-0.305288,0.699908,0.052858,-0.335955,-0.305556,1.216790,Delaware
8,FL,1.051071,-0.165507,4.096954e-01,-0.367112,-0.153066,0.242697,-0.108979,0.005788,0.568973,-0.261065,0.625003,-0.485291,0.016469,0.369084,-0.341335,Florida
9,GA,-0.175739,-0.057487,8.239138e-02,0.418938,-0.535657,0.125227,0.366283,-0.170931,0.371589,-0.285531,0.293679,-0.136114,0.535139,-0.425554,-0.383168,Georgia


In [62]:
fips_rows = []
for i, row in leadxpop_states_by_year.iterrows():
    fips_row = {
        'state_abrv': row['state_abrv']
    }
    fips_rows.append(fips_row)

fips_rows[:5]

[{'state_abrv': 'AK'},
 {'state_abrv': 'AL'},
 {'state_abrv': 'AR'},
 {'state_abrv': 'AZ'},
 {'state_abrv': 'CA'}]

In [63]:
# code from https://pypi.org/project/addfips/
import addfips

af = addfips.AddFIPS()
for row in fips_rows:
    af.add_state_fips(row, state_field='state_abrv')
    
fips_rows[:5]

[{'state_abrv': 'AK', 'fips': '02'},
 {'state_abrv': 'AL', 'fips': '01'},
 {'state_abrv': 'AR', 'fips': '05'},
 {'state_abrv': 'AZ', 'fips': '04'},
 {'state_abrv': 'CA', 'fips': '06'}]

In [64]:
state_fips = pd.DataFrame(fips_rows)
state_fips

,state_abrv,fips
0,AK,02
1,AL,01
2,AR,05
3,AZ,04
4,CA,06
5,CO,08
6,CT,09
7,DE,10
8,FL,12
9,GA,13


In [65]:
leadxpop_states_by_year = leadxpop_states_by_year.merge(state_fips, on='state_abrv')
# leadxpop_states_by_year.drop('State', axis=1, inplace=True)
leadxpop_states_by_year

,state_abrv,2005_lead_mg/L_x_population_served,2006_lead_mg/L_x_population_served,2007_lead_mg/L_x_population_served,2008_lead_mg/L_x_population_served,2009_lead_mg/L_x_population_served,2010_lead_mg/L_x_population_served,2011_lead_mg/L_x_population_served,2012_lead_mg/L_x_population_served,2013_lead_mg/L_x_population_served,2014_lead_mg/L_x_population_served,2015_lead_mg/L_x_population_served,2016_lead_mg/L_x_population_served,2017_lead_mg/L_x_population_served,2018_lead_mg/L_x_population_served,2019_lead_mg/L_x_population_served,2020_lead_mg/L_x_population_served,fips
0,AK,3.839868e-05,0.000011,0.000040,4.388179e-05,4.060238e-05,3.154968e-05,3.994240e-05,2.673924e-05,1.940794e-05,1.452318e-05,1.179923e-05,7.810501e-06,4.439066e-06,1.031811e-05,3.887469e-06,5.004808e-06,02
1,AL,1.669901e-05,0.000021,0.000016,1.936102e-05,2.194802e-05,2.008235e-05,2.028438e-05,1.915712e-05,1.540450e-05,1.622684e-05,1.654256e-05,1.293906e-06,1.656337e-06,1.591977e-06,2.525043e-06,1.874503e-06,01
2,AR,2.186938e-05,0.000029,0.000023,7.146147e-05,2.079913e-05,4.225000e-05,1.888493e-05,2.844444e-05,2.402500e-04,9.576516e-06,9.506250e-05,5.184000e-05,6.676905e-05,1.506198e-04,4.144141e-05,6.524024e-06,05
3,AZ,1.403565e-05,0.001774,0.000007,9.431723e-06,1.445596e-05,5.941858e-06,1.358126e-04,1.909822e-05,7.077646e-06,1.760579e-05,1.309965e-05,4.125044e-06,3.585393e-06,6.666724e-06,2.951033e-06,1.707845e-05,04
4,CA,5.407313e-04,0.000022,0.000052,5.344904e-06,4.744534e-06,4.057132e-06,5.725013e-06,2.375832e-06,2.311946e-06,2.343539e-06,1.191498e-06,1.335598e-06,7.057744e-07,7.185656e-07,1.059184e-06,3.703024e-06,06
5,CO,2.922517e-05,0.000017,0.000015,1.712603e-05,2.959763e-05,4.038822e-05,2.249748e-05,5.600028e-05,4.153086e-05,1.426989e-05,5.228402e-05,2.575563e-05,1.111530e-05,1.175839e-05,1.108926e-05,9.031774e-06,08
6,CT,6.539887e-07,0.000008,0.000009,1.446965e-06,2.348556e-06,1.546569e-06,4.642151e-07,1.163005e-06,4.588670e-07,9.850562e-07,1.595279e-06,6.337814e-07,7.180332e-07,8.307866e-07,8.327165e-07,1.449616e-06,09
7,DE,1.897086e-05,0.000009,0.000061,1.764000e-05,2.828306e-05,3.647876e-05,1.273136e-04,1.865006e-05,1.475840e-05,9.922500e-06,7.910156e-06,1.247242e-05,4.536900e-06,4.231837e-06,6.201563e-07,5.725765e-06,10
8,FL,4.243600e-06,0.000031,0.000028,1.759514e-05,1.555316e-05,1.459240e-05,8.917878e-06,9.798663e-06,6.276624e-06,5.238705e-06,6.093845e-06,1.026784e-05,7.503048e-06,5.604914e-06,6.172920e-06,4.996388e-06,12
9,GA,1.083507e-05,0.000012,0.000010,1.444000e-05,7.018009e-06,1.740510e-05,1.781104e-05,9.262355e-06,8.692212e-06,4.818334e-06,4.415835e-06,1.491504e-05,6.404567e-06,6.460312e-06,4.658403e-06,3.729727e-06,13


In [66]:
fips_rows = []
for i, row in copperxpop_states_by_year.iterrows():
    fips_row = {
        'state_abrv': row[('state_abrv')]
    }
    fips_rows.append(fips_row)

fips_rows[:5]

[{'state_abrv': 'AK'},
 {'state_abrv': 'AL'},
 {'state_abrv': 'AR'},
 {'state_abrv': 'AZ'},
 {'state_abrv': 'CA'}]

In [67]:

af = addfips.AddFIPS()
for row in fips_rows:
    af.add_state_fips(row, state_field='state_abrv')
    
state_fips = pd.DataFrame(fips_rows)
state_fips

,state_abrv,fips
0,AK,02
1,AL,01
2,AR,05
3,AZ,04
4,CA,06
5,CO,08
6,CT,09
7,DE,10
8,FL,12
9,GA,13


In [68]:
copperxpop_states_by_year = copperxpop_states_by_year.merge(state_fips, on='state_abrv')
# copper_states_by_year.drop('State', axis=1, inplace=True)
copperxpop_states_by_year

,state_abrv,2005_copper_mg/L_x_population_served,2006_copper_mg/L_x_population_served,2007_copper_mg/L_x_population_served,2008_copper_mg/L_x_population_served,2009_copper_mg/L_x_population_served,2010_copper_mg/L_x_population_served,2011_copper_mg/L_x_population_served,2012_copper_mg/L_x_population_served,2013_copper_mg/L_x_population_served,2014_copper_mg/L_x_population_served,2015_copper_mg/L_x_population_served,2016_copper_mg/L_x_population_served,2017_copper_mg/L_x_population_served,2018_copper_mg/L_x_population_served,2019_copper_mg/L_x_population_served,2020_copper_mg/L_x_population_served,fips
0,AK,5.342840,6.615506,3.892729,6.932689e+00,487032.086747,4.448005,56057.722049,5.994369,5.532427,8.104374,3.298497,5.695816,4.831754,5.030408,5.924660,4.972900,02
1,AL,4.694444,7.971941,17.222500,5.522500e+00,5.872544,3.312400,11.971600,12.752041,14.364100,10.017225,585225.000000,5.683456,4.633256,11.639469,2.480625,8.179600,01
2,AR,3.572100,7.971941,57.154079,7.150691e+04,1681.000000,7.125129,67.197458,52.741406,54.896331,6.002500,881.455120,9.187919,23.968807,7.828961,7.051265,2.131600,05
3,AZ,3.240000,4.401604,2.711511,5.681722e+00,3.694084,7.193124,4.131056,10.868011,543.356100,5.472059,1.988100,3.006756,9.922500,10.176100,4.000000,34.339600,04
4,CA,22.634942,7.614941,181.564494,4.388944e+00,4.012009,5.513285,13.406408,41.893394,4.873387,4.363338,5.280608,37.069076,4.112743,4.834087,5.293055,4.600837,06
5,CO,117.290138,5.895595,2.777778,8.497225e+00,4.462656,4.305625,4.629539,3.496900,4.662900,4.513854,3.908225,3.825284,3.130326,4.279727,3.841600,3.506256,08
6,CT,6.406323,4.854923,6.037259,4.154317e+00,4.247938,7.728400,5.815518,5.462844,7.562500,8.556708,6.131814,4.419698,7.338090,5.445861,9.138529,3.559378,09
7,DE,9.679654,669.619129,57.154079,3.956121e+00,4.508403,8.101139,13.557124,10.908865,6.718464,8.829388,6.133878,10.427028,10.978178,7.290000,5.062500,11.222500,10
8,FL,2.706025,5.550249,4.631642,6.529204e+00,4.132252,3.499743,4.349121,3.875156,3.897587,6.115208,4.518742,7.342971,3.779492,3.841737,5.259662,3.464355,12
9,GA,7.707286,6.352815,5.987613,6.480941e+00,9.196056,4.270127,4.804864,6.564805,5.442677,7.465118,5.333598,6.899961,5.960779,9.150625,5.256539,3.242400,13


In [69]:
# copperxpop_per_change = copperxpop_states_by_year.merge(state_fips, on='state_abrv')
# leadxpop_per_change = leadxpop_states_by_year.merge(state_fips, on='state_abrv')
# # copper_states_by_year.drop('State', axis=1, inplace=True)
# copperxpop_per_change

In [70]:
leadxpop_pct_change.columns

Index(['state_abrv', '2006_lead_mg/L_x_population_served',
       '2007_lead_mg/L_x_population_served',
       '2008_lead_mg/L_x_population_served',
       '2009_lead_mg/L_x_population_served',
       '2010_lead_mg/L_x_population_served',
       '2011_lead_mg/L_x_population_served',
       '2012_lead_mg/L_x_population_served',
       '2013_lead_mg/L_x_population_served',
       '2014_lead_mg/L_x_population_served',
       '2015_lead_mg/L_x_population_served',
       '2016_lead_mg/L_x_population_served',
       '2017_lead_mg/L_x_population_served',
       '2018_lead_mg/L_x_population_served',
       '2019_lead_mg/L_x_population_served',
       '2020_lead_mg/L_x_population_served', 'State'],
      dtype='object')

In [71]:
copperxpop_pct_change.columns

Index(['state_abrv', '2006_copper_mg/L_x_population_served',
       '2007_copper_mg/L_x_population_served',
       '2008_copper_mg/L_x_population_served',
       '2009_copper_mg/L_x_population_served',
       '2010_copper_mg/L_x_population_served',
       '2011_copper_mg/L_x_population_served',
       '2012_copper_mg/L_x_population_served',
       '2013_copper_mg/L_x_population_served',
       '2014_copper_mg/L_x_population_served',
       '2015_copper_mg/L_x_population_served',
       '2016_copper_mg/L_x_population_served',
       '2017_copper_mg/L_x_population_served',
       '2018_copper_mg/L_x_population_served',
       '2019_copper_mg/L_x_population_served',
       '2020_copper_mg/L_x_population_served', 'State'],
      dtype='object')

In [72]:
lead_mean = leadxpop_states_by_year.mean(axis=1)
copper_mean = copperxpop_states_by_year.mean(axis=1)
lead_mean = leadxpop_per_change.mean(axis=1)
copper_mean = copperxpop_per_change.mean(axis=1)

<ipython-input-72-196dbecd3e75>:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  lead_mean = leadxpop_states_by_year.mean(axis=1)
<ipython-input-72-196dbecd3e75>:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  copper_mean = copperxpop_states_by_year.mean(axis=1)


NameError: name 'leadxpop_per_change' is not defined

In [ ]:
copperxpop_states_by_year = copperxpop_states_by_year.sort_values(by='fips')
leadxpop_states_by_year = leadxpop_states_by_year.sort_values(by='fips')
copperxpop_per_change = copperxpop_states_by_year.sort_values(by='fips')
leadxpop_per_change = leadxpop_states_by_year.sort_values(by='fips')

In [ ]:
copperxpop_pct_change.to_csv('health_issues_per_change/copperxpop_pct_change.csv')
leadxpop_pct_change.to_csv('health_issues_per_change/leadxpop_pct_change.csv')

In [ ]:
copperxpop_states_by_year.to_csv('./datasets/copperxpop_states_by_year.csv')
leadxpop_states_by_year.to_csv('./datasets/leadxpop_states_by_year.csv')

In [ ]:
lead_states_by_year.to_csv('./datasets/lead_states_by_year.csv')
copper_states_by_year.to_csv('./datasets/copper_states_by_year.csv')

In [ ]:
copper.to_csv('./datasets/water_copper.csv')
lead.to_csv('./datasets/water_lead.csv')

In [ ]:
high_lead = lead[lead['Sample Measure (mg/L)'] > 0.015]

high_lead.groupby(['Primacy Agency Code', '']).agg({'Sample Measure (mg/L)': ['count', 'mean', 'min', 'max', 'sum'],
                                           'Population Served Count': ['count', 'mean', 'min', 'max', 'sum'],
                                           'Sampling Start Date': ['mean', 'min', 'max'],
                                           'Sampling End Date': ['mean', 'min', 'max']})

In [ ]:
high_lead[(high_lead['Primacy Agency Code'] == 'OH') & (high_lead['Population Served Count'] > 0)].sort_values(by='Sample Measure (mg/L)', ascending=False)

In [ ]:
# code help from https://medium.com/@erikgreenj/mapping-us-states-with-geopandas-made-simple-d7b6e66fa20d

usa = gpd.read_file('./states_21basic/states.shp')

In [ ]:
# lead_by_state = lead.groupby('Primacy Agency Code')['Sample Measure (mg/L)'].mean()
# lead_by_state = lead_by_state[usa.STATE_ABBR]
# lead_by_state

In [ ]:
# # https://plotly.com/python/choropleth-maps/

# from urllib.request import urlopen
# import json
# with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-states-fips.json') as response:
#     counties = json.load(response)

# import pandas as pd
# # df = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/fips-unemp-16.csv",
# #                    dtype={"fips": str})

# import plotly.express as px

# fig = px.choropleth(copper_by_states, geojson=states, locations='fips', color=('Sample Measure (mg/L)'),
#                            color_continuous_scale='darkmint',
#                            range_color=(0, 12),
#                            scope="usa",
#                            labels={'Sample Measure (mg/L)':'mean'}
#                           )
# fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
# fig.show()

In [ ]:
lead_by_states[('Sample Measure (mg/L)', 'mean')]

In [ ]:
# code from https://plotly.com/python/choropleth-maps/

import plotly.graph_objects as go


fig = go.Figure(data=go.Choropleth(
    locations=lead_states_by_year['state_abrv'], # Spatial coordinates
    z = lead_states_by_year.mean(axis=1), # Data to be color-coded
    locationmode = 'USA-states', # set of locations match entries in `locations`
    colorscale = 'Reds',
    colorbar_title = "Mean",
#     hovertext=lead_states_by_year['2006_Lead Sample Measure (mg/L)'],
    marker_line_color='white'
))

fig.update_layout(
    title_text = 'Lead Sample Measure (mg/L) Mean by State',
    geo_scope='usa', # limit map scope to USA
)


fig.show()

In [ ]:
import plotly.graph_objects as go


fig = go.Figure(data=go.Choropleth(
    locations=copper_states_by_year['state_abrv'], # Spatial coordinates
    z = copper_states_by_year.mean(axis=1), # Data to be color-coded
    locationmode = 'USA-states', # set of locations match entries in `locations`
    colorscale = 'Blues',
    colorbar_title = "Mean",
#     hovertext=lead_states_by_year['2006_Lead Sample Measure (mg/L)'],
    marker_line_color='white'
))

fig.update_layout(
    title_text = 'Copper Sample Measure (mg/L) Mean by State',
    geo_scope='usa', # limit map scope to USA
)


fig.show()